In [16]:
# ollama pull llama3.2
import json

import numpy as np

from ollama import chat
from ollama import ChatResponse

from qdrant_client import QdrantClient, models
from fastembed import TextEmbedding

from tqdm.auto import tqdm


EMBEDDING_DIMENSIONALITY = 512
MODEL_HANDLE             = "jinaai/jina-embeddings-v2-small-en"
COLLECTION_NAME          = "homework-rag-vector-search"


```
docker pull qdrant/qdrant

docker run -p 6333:6333 -p 6334:6334 \
   -v "$(pwd)/qdrant_storage:/qdrant/storage:z" \
   qdrant/qdrant
```

In [11]:
text = "I just discovered the course. Can I join now?"

In [12]:
embedding_model = TextEmbedding(
    model_name=MODEL_HANDLE,
    dimensionality=EMBEDDING_DIMENSIONALITY,
)

In [13]:
embeddings_generator = embedding_model.embed([text])
embeddings_list = list(embeddings_generator)

In [17]:
np.min(embeddings_list[0])

np.float64(-0.11726373885183883)

In [18]:
len(embeddings_list[0])

512

In [19]:
text2 = "Can I still join the course after the start date?"

embeddings_generator = embedding_model.embed([text, text2])
embeddings_list = list(embeddings_generator)

In [20]:
embeddings_list[0].dot(embeddings_list[1])

np.float64(0.9008528895674548)

In [ ]:
query = "I just discovered the course. Can I join now?"

documents = [
  {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

texts = [doc['text'] for doc in documents]

embeddings_generator = embedding_model.embed([query]+texts)
embeddings_list = list(embeddings_generator)

In [22]:
similarities = [embeddings_list[0].dot(embeddings_list[i]) for i in range(1, len(embeddings_list))]

In [ ]:
similarities # 1

[np.float64(0.7629684696540238),
 np.float64(0.8182378150042889),
 np.float64(0.8085397398734189),
 np.float64(0.7133079015686243),
 np.float64(0.7304499234333598)]

In [ ]:
full_text = [doc['question'] + ' ' + doc['text'] for doc in documents]

embeddings_generator = embedding_model.embed([query]+full_text)
embeddings_list = list(embeddings_generator)

similarities = [embeddings_list[0].dot(embeddings_list[i]) for i in range(1, len(embeddings_list))]
similarities # 0
# the answer changed because we are using the full text now, with more information and closer to the query

[np.float64(0.8514543236908068),
 np.float64(0.8436594159113068),
 np.float64(0.8408287048502558),
 np.float64(0.7755157969663908),
 np.float64(0.8086007795043938)]

In [ ]:
np.min([model["dim"] for model in TextEmbedding.list_supported_models()])

np.int64(384)

In [30]:
import json

EMBEDDING_DIMENSIONALITY = 384

for model in TextEmbedding.list_supported_models():
    if model["dim"] == EMBEDDING_DIMENSIONALITY:
        print(json.dumps(model, indent=2))



{
  "model": "BAAI/bge-small-en",
  "sources": {
    "hf": "Qdrant/bge-small-en",
    "url": "https://storage.googleapis.com/qdrant-fastembed/BAAI-bge-small-en.tar.gz",
    "_deprecated_tar_struct": true
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.13,
  "additional_files": [],
  "dim": 384,
  "tasks": {}
}
{
  "model": "BAAI/bge-small-en-v1.5",
  "sources": {
    "hf": "qdrant/bge-small-en-v1.5-onnx-q",
    "url": null,
    "_deprecated_tar_struct": false
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.067,
  "additional_files": [],
  "dim": 384,
  "tasks": {}
}
{
  "model": "snowflake/snowflake-arctic-embed-xs",
  "sou

In [31]:
EMBEDDING_DIMENSIONALITY = 384
MODEL_HANDLE             = "BAAI/bge-small-en"
COLLECTION_NAME          = "homework-rag-vector-search"

In [37]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [38]:
qd_client = QdrantClient("http://localhost:6333") # connecting to local Qdrant instance
qd_client.delete_collection(collection_name=COLLECTION_NAME)

True

In [39]:
qd_client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

True

In [40]:
points = []

for id, doc in enumerate(documents):
    text = doc["question"] + " " + doc["text"]
    vector = models.Document(text=text, model=MODEL_HANDLE)

    point = models.PointStruct(
        id=id,
        vector=vector, # embed text locally with "jinaai/jina-embeddings-v2-small-en" from FastEmbed
        payload=doc
    )
    points.append(point)

qd_client.upsert(
    collection_name=COLLECTION_NAME,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [41]:
def vector_search(query, 
                 course="mlops-zoomcamp", 
                 limit=1):

    query_points = qd_client.query_points(
        collection_name=COLLECTION_NAME,
        query=models.Document(
            text=query,
            model=MODEL_HANDLE
        ),
        # query_filter=models.Filter( # filter by course name
        #     must=[
        #         models.FieldCondition(
        #             key="course",
        #             match=models.MatchValue(value=course)
        #         )
        #     ]
        # ),
        limit=limit, # top closest matches
        with_payload=True # to get metadata in the results
    )

    return query_points
    # results = []
    
    # for point in query_points.points:
    #     results.append(point.payload)
    
    # return results

In [43]:
vector_search("I just discovered the course. Can I join now?").points

[ScoredPoint(id=14, version=0, score=0.8703172, payload={'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'question': 'The course has already started. Can I still join it?', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None)]